In [ ]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

f = open('../../Latin/Pliny/opensource/PlinyNH.xml')
ltext = f.read()
f.close()
f = open('../../Latin/Pliny/opensource/pliny.maj.nh_eng.xml')
etext = f.read()
f.close()

In [7]:
lat_books = re.split(r'<div0 type="book" n="\d\d?\d?">', ltext)

eng_books = re.split(r'<div1 type="book" n="\d\d?\d?">', etext)

In [8]:
lat_books = lat_books[1:]
eng_books = eng_books[1:]

In [47]:
lat_sects = []
eng_sects = []

for b in range(len(lat_books)):
    lat_sects.append(re.split(r'<div1 type="chapter" n="\d\d?\d?">', lat_books[b]))
    eng_sects.append(re.split(r'<div2 type="chapter" n="\d\d?\d?">', eng_books[b]))

In [37]:
len(re.split(r'<p>', lat_sects[1][0]))

1

In [38]:
len(re.split(r'<p>', eng_sects[1][0]))

2

In [48]:
len(lat_sects[1][0])

23

In [52]:
eng_sects[1][1]

'\n<head>CHAP. 1. (1.)&mdash;WHETHER THE WORLD BE FINITE, AND\nWHETHER THERE BE MORE THAN ONE WORLD.</head>\n\n<p>The world<note anchored="yes" place="foot">"Mundus." In translating from one language into another, it is\nproper, as a general principle, always to render the same word in the\noriginal by the same word in the translation. But to this rule there are\ntwo exceptions; where the languages do not possess words which precisely correspond, and where the original author does not always use the\nsame word in the same sense. Both these circumstances, I apprehend,\napply to the case in question. The term <hi rend="ital">Mundus</hi> is used by <placeName key="tgn,2588096">Pliny</placeName>,\nsometimes to mean <hi rend="ital">the earth</hi> and its immediate appendages, the visible\nsolar system; and at other times <hi rend="ital">the universe;</hi> while I think we may\nventure to assert, that in some instances it is used in rather a vague\nmanner, without any distinct reference to e

In [51]:
lat_sects[1][1]

'\n<milestone unit="para" n="1"/>\n<p>\n</p>\n<p>\nmundum et hoc quodcumque nomine alio caelum appellare libuit, cuius circumflexu degunt cuncta, numen esse credi par est, aeternum, inmensum, neque genitum neque interiturum umquam. huius extera indagare nec interest hominum nec capit humanae coniectura mentis. sacer est, aeternus, immensus, totus in toto, immo vero ipse totum, infinitus ac finito similis, omnium rerum certus et similis incerto, extra intra cuncta conplexus in se, idemque rerum naturae opus et rerum ipsa natura. furor est mensuram eius animo quosdam agitasse atque prodere ausos, alios rursus occasione hinc sumpta aut hi<hi rend="italics">c</hi> data innumerabiles tradidisse mundos, ut totidem rerum naturas credi oporteret aut, si una omnes incubaret, totidem tamen\n<pb />\nsoles totidemque lunas et cetera etiam in uno et inmensa et innumerabilia sidera, quasi non eaedem quaestiones semper in termino cogitationi sint occursurae desiderio finis alicuius aut, si haec infin